In [328]:
import pandas as pd
import jsonpickle
import numpy as np
import matplotlib.pyplot as plt
import os, os.path
import dateutil.parser
from datetime import datetime

Constants

In [329]:
FILE_AS_ROOT = False
SELF_PATH = os.getcwd()#os.path.dirname(os.path.abspath(__file__))
PATH_TO_ROOT = os.path.join(SELF_PATH,'../../../../../../' if FILE_AS_ROOT else '')
PATH_TO_WRITE_CSV = os.path.join(PATH_TO_ROOT,'../')
PATH_TO_JSON_STATE = os.path.join(PATH_TO_ROOT,'../latest_state.json')

Given path to a pickle state,will return pickle object

In [330]:
def get_pickle(path_to_file):
    with open(path_to_file, 'r') as f:
        return jsonpickle.decode(f.read())

Given a pickgle object will return how many incomplete, complete images have been tagged along with unix epoch time and date time stamp

In [331]:
def how_many_tagged(pickle_file):
    
    done_tagging_count=len(pickle_file.finished_tagged_queue)
    tagged_but_not_done_count=0
    
    for image in pickle_file.pending_images_queue:
        if len(image.get_taggers()) > 0:
            tagged_but_not_done_count+=1
    
    return pd.DataFrame([{ 
        'not_done':tagged_but_not_done_count,
        'done':done_tagging_count,
        'tagged_ratio': tagged_but_not_done_count/(tagged_but_not_done_count+done_tagging_count)
     }])

In [332]:
pickle = get_pickle(PATH_TO_JSON_STATE)
how_many_tagged(pickle)

,not_done,done,tagged_ratio
0,117,248,0.320548


In [333]:
finished_images = pickle.finished_tagged_queue
partical_tagged = pickle.pending_images_queue

In [364]:
def gen_df(images,history = True):
    image_tag_df = pd.DataFrame(dtype = int)

    for image in images:

        tagger_list = list(image.get_taggers())
        image_id = image.get_rel_path()

        dict_row ={
            'image_id':image_id
            # 'tagging_start':image.stats_tagging_start,
            # 'tagging_end':image.stats_tagging_stop,
            # 'tagging_session_time':image.stats_tag_elapsed_session,
            # 'tagging_assingment_time':image.stats_tag_elapsed_assigned
        }
        
        # View all tags of an image, not final
        if history:
            for tagger in tagger_list:
                # Get all the tags tagged by this tagger
                list_of_tags = (image.get_tags(tagger))
                
                # Add those tags and the tagger ID to the dictionary that will be our row
                dict_row.update(list_of_tags)
                dict_row.update({
                    'tagger':tagger,
                    'tagging_start':image.stats_tagging_start[tagger],
                    'tagging_end':image.stats_tagging_stop[tagger],
                    'elapsed_session_time':image.stats_tag_elapsed_session[tagger],
                    'total_assingment_time':image.stats_tag_elapsed_assigned[tagger]
                })
                
                # Combine the dictionary row as a dataframe into the image tag dataframe
                tag_row_df = pd.DataFrame([dict_row],dtype = int) 
                image_tag_df = pd.concat([image_tag_df,tag_row_df],sort=True , ignore_index = True)
        else:
            final_tags_df = image.final_tags 
            dict_row.update(final_tags_df)
            
            
            starting_times = []
            ending_times = []
            assinged_times =[]
            session_times =[]

            for tagger in image.stats_tagging_start:
                starting_times.append(image.stats_tagging_start[tagger])
            
            for tagger in image.stats_tagging_stop:
                ending_times.append(image.stats_tagging_stop[tagger])
            
            for tagger in image.stats_tag_elapsed_assigned:
                assinged_times.append(image.stats_tag_elapsed_assigned[tagger])
            
            for tagger in image.stats_tag_elapsed_session:
                session_times.append(image.session_times[tagger])

            dict_row.update({
                'start_time':min(starting_times),
                'end_time':max(ending_times),
                'assinged_time':min(assinged_times),
                'elapsed_time':max(ending_times)-min(starting_times)
            })

            tag_row_df = pd.DataFrame([dict_row],dtype = int) 

            image_tag_df = pd.concat([image_tag_df,tag_row_df],sort=True, ignore_index = True)
    return image_tag_df   

In [365]:
#with_history_tags = gen_df()
final_tags =gen_df(finished_images,False)

In [336]:
final_tags.head()

,development,image_id,impact,ocean,terrain_inland,terrain_marsh,terrain_river,terrain_sandy_coastline,terrain_undefined,washover
0,0.0,jpgs/S26047238.jpg,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0
1,1.0,jpgs/P26057146.jpg,2.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0
2,1.0,jpgs/P26054308.jpg,2.0,NaN,NaN,NaN,NaN,1.0,NaN,0.0
3,1.0,jpgs/P26050957.jpg,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0
4,0.0,jpgs/P26057962.jpg,3.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0


In [337]:
def get_all_taggers(images):
    taggers_df = pd.DataFrame()

    for image in images:
        taggers = list(image.get_taggers())
        for tagger in taggers:
            tagger_as_df = pd.DataFrame([tagger]) 
            taggers_df = pd.concat([taggers_df,tagger_as_df],sort=True , ignore_index = True)

    return taggers_df
all_taggers = get_all_taggers(finished_images)
all_taggers.drop_duplicates(inplace = True)

In [338]:
# all_taggers

In [339]:
final_tags.replace([False,True],[0,1], inplace=True)

final_tags[['washover','impact','development','ocean','terrain_inland','terrain_marsh','terrain_river','terrain_sandy_coastline','terrain_undefined']] = final_tags[['washover','impact','development','ocean','terrain_inland','terrain_marsh','terrain_river','terrain_sandy_coastline','terrain_undefined']].fillna(0.0).astype(int)
final_tags

,development,image_id,impact,ocean,terrain_inland,terrain_marsh,terrain_river,terrain_sandy_coastline,terrain_undefined,washover
0,0,jpgs/S26047238.jpg,0,0,0,0,0,0,1,0
1,1,jpgs/P26057146.jpg,2,0,0,0,0,1,0,0
2,1,jpgs/P26054308.jpg,2,0,0,0,0,1,0,0
3,1,jpgs/P26050957.jpg,0,0,0,0,0,0,1,0
4,0,jpgs/P26057962.jpg,3,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
243,1,jpgs/C26050368.jpg,0,0,0,0,1,0,1,0
244,0,jpgs/P26058354.jpg,3,0,0,0,0,1,0,1
245,0,jpgs/P26051780.jpg,0,0,0,0,1,0,1,0
246,1,jpgs/C26049792.jpg,0,0,1,0,0,0,1,0


In [340]:
#final_tags.to_csv(os.path.join(PATH_TO_WRITE_CSV,'image_tags.csv'))